In [ ]:
!kill -9 -1 #deco/reco le Colab

In [ ]:
!pip install transformers==4.24.0
!pip install sentencepiece

from google.colab import drive
drive.mount('/content/drive')

import torch
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd

In [2]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = """
 Louis Pouzin\nSummary: Louis Pouzin est un ingénieur français en informatique né le 20 avril 1931 à Chantenay-Saint-Imbert (Nièvre) dans une famille modeste. Il a créé et développé au début des années 1970 le Réseau Cyclades, à base de datagramme pur, et a contribué au développement des réseaux à commutation de paquets, précurseurs d'Internet. Ses travaux, et ceux de ses collaborateurs du projet  Cyclades Hubert Zimmermann, Gérard Le Lann et Michel Elie, ont eu une influence reconnue par Vint Cerf dans une de ses premières spécifications du protocole de transport d'Internet TCP.\n\nPage: Cyclades (réseau)\nSummary: Cyclades est un réseau d'ordinateurs français, concurrent et partenaire du réseau américain Arpanet, qui a le premier utilisé les datagrammes. Lancé en 1972 sous la direction Louis Pouzin, le réseau est abandonné en 1978 après l'ouverture du réseau public à commutation de paquets Transpac. \nEn lien avec le réseau américain Arpanet et le réseau britannique du NPL, les concepts développés au sein du réseau Cyclades ont influencé les travaux de développement de l'Internet en inspirant sa suite de protocoles.\n\nPage: Le Pouzin\nSummary: Le Pouzin est une commune française située dans le département de l'Ardèche en région Auvergne-Rhône-Alpes.\nLes habitants sont appelés les Pouzinois.
"""

model_name = "csebuetnlp/mT5_multilingual_XLSum"

In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


CPU times: user 11.2 s, sys: 6.63 s, total: 17.8 s
Wall time: 60 s


In [3]:
%%time
input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

Notre série de lettres de journalistes africains a analysé l'histoire de Louis Pouzin.
CPU times: user 17.3 s, sys: 10.3 ms, total: 17.3 s
Wall time: 19.7 s


# Comparaison avec CUDA

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

In [8]:
%%time
input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)

input_ids = input_ids.to("cuda")["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

Notre série de lettres de journalistes africains a analysé l'histoire de Louis Pouzin.
CPU times: user 1.9 s, sys: 33.3 ms, total: 1.94 s
Wall time: 3.63 s


# Avec plus d'éléments en entrée:

In [10]:
article_text = """

Louis Pouzin
Louis Pouzin en 2013.
Biographie Naissance 	
20 avril 1931 (90 ans)
Chantenay-Saint-Imbert
Nationalité 	
Français
Formation 	
École polytechnique (1950-1953)
Activités 	
Informaticien, ingénieur
Autres informations A travaillé pour 	
Centre national d'études des télécommunications (1980-1989), Inria (1971-1980), Bull (1957-1967), Société industrielle des téléphones (1953-1957)
Domaine 	
Informatique
Distinctions 	
Liste détaillée
Prix SIGCOMM (en) (1997)
IEEE Internet Award (2001)
Temple de la renommée d'Internet (2012)
Queen Elizabeth Prize for Engineering (2013)
Officier de la Légion d'honneur‎ (2018)
Œuvres principales
Datagramme

modifier - modifier le code - modifier Wikidata

Louis Pouzin est un ingénieur français en informatique né le 20 avril 1931 à Chantenay-Saint-Imbert (Nièvre) dans une famille modeste[1]. Il a créé et développé au début des années 1970, le Réseau Cyclades, à base de datagramme pur, et a contribué au développement des réseaux à commutation de paquets, précurseurs d'Internet. Ses travaux, et ceux de ses collaborateurs du projet Cyclades Hubert Zimmermann, Gérard Le Lann et Michel Elie, ont eu une influence reconnue par Vint Cerf dans une de ses premières spécifications du protocole de transport d'Internet TCP[2].
Sommaire

1 Biographie

1.1 Polytechnicien informaticien ?
1.2 Temps partagé

1.2.1 MIT-Boston
1.2.2 Météo-France
1.3 Réseau Cyclades

1.3.1 Réalisation
1.3.2 Datagrammes vs Transpac
1.3.3 Fin de l'expérimentation
1.4 Après Cyclades
2 Distinctions

2.1 Prix
2.2 Décorations
3 Notes et références

3.1 Notes
3.2 Références
4 Voir aussi

4.1 Bibliographie
4.2 Articles connexes
4.3 Liens externes
Biographie[modifier | modifier le code]
Polytechnicien informaticien ?[modifier | modifier le code]

En 1953, Louis Pouzin, ancien élève de l'École polytechnique (promotion 1950)[3],[Note 1] , est embauché à la Compagnie Industrielle des Téléphones (CIT, qui deviendra plus tard CIT-Alcatel). Après avoir lu un article du quotidien Le Monde parlant de calculateurs[5], il répond, en 1957, à une offre d'emploi chez Bull. Il y dirige le service qui met au point les programmes pour les clients qui ont acheté un ordinateur Bull, programmes qu'une équipe de techniciens transforme en cartes perforées. Ses camarades de l'X ne comprennent pas pourquoi Louis Pouzin s'est fourvoyé dans un domaine et une fonction aussi peu intéressants : la programmation est une tâche de technicien et les calculateurs ne servent qu'à la comptabilité. Il fait partie des trois seuls diplômés de sa promotion à s'intéresser à cette nouvelle discipline qui deviendra l'informatique.
Temps partagé[modifier | modifier le code]
MIT-Boston[modifier | modifier le code]

C'est alors qu'il rencontre Fernando J. Corbató, professeur au MIT, qui lui parle de son projet de recherche baptisé Multi Access Computer. Il s'agit de permettre à plusieurs usagers d'utiliser un même ordinateur en temps partagé (Time sharing). Le concept séduit Louis Pouzin qui se laisse inviter au MIT. De janvier 1963[5] à juin 1965, il est l'un des six programmeurs du Compatible Time Sharing System (CTSS). Il conçoit une bibliothèque d'utilitaires, et différents éléments du programme. Il écrit également un programme appelé RUNCOM qui permet l'exécution de commandes contenues dans un fichier, et que l'on peut considérer comme l'ancêtre de l'interpréteur de commandes et des shell scripts. Comme le CTSS évolue vers Multics, un système d'exploitation plus ambitieux pouvant accueillir simultanément des centaines d'utilisateurs, il définit les spécifications de l'interpréteur de commandes baptisé "shell" (coquille) qui sera écrit par Glenda Schroeder au MIT après son retour en France.
Météo-France[modifier | modifier le code]

En France, Louis Pouzin rejoint Bull qui, entre temps, a été acquis par General Electric. Il donne des conférences afin de présenter les concepts qu'il a mis en œuvre aux États-Unis. L'ordinateur Bull-GE 600 est convoité par Météo-France qui demande à Louis Pouzin de modifier le système d'exploitation pour permettre des usages multiples. Mais GE décide brusquement de ne plus commercialiser le GE-600 en Europe. À la suite d'un appel d'offre, Météo-France fait affaire avec Control Data et demande à Louis Pouzin d'adapter le système d'exploitation du nouvel ordinateur.

En 1967, Louis Pouzin quitte Bull et se fait embaucher par une jeune société, SACS, qui donnera naissance à la Société d'économie et de mathématiques appliquées (SEMA, future Atos). Il conçoit le système d'exploitation qui permet de travailler en temps partagé et qui restera quinze ans en service à Météo-France. Le but du projet était d'automatiser les cartes géographiques de l'organisme (à l'époque tout était fait à la main).

Louis Pouzin participe entre-temps à la Conférence d'Edimbourg, du 5 août 1968, où Donald Davies fait une présentation sur l'importance des questions de réseau[6] pour le temps partagé. En 1969, il visite les universitaires du réseau ARPANET américain (Philadelphie, Los Angeles – UCLA, Rand, SDC et IBM –, San Francisco – SRI, Berkeley – et à nouveau le MIT à Boston)[7]. ARPANET est le premier réseau d'ordinateurs partagés à l'échelle d'un pays. Pouzin est sensible aux difficultés rencontrées par ses collègues américains, en particulier le problème des adresses physiques, de la trop grande dépendance envers les cartes physiques et de la faible efficacité du protocole "host to host"< [réf. nécessaire].

En 1970, le projet chez Météo France est terminé. Louis Pouzin cherche un autre projet d'envergure. Il répond à une petite annonce pour un poste de chef du service informatique chez le constructeur automobile Simca qui est équipé d'un ordinateur IBM qu'il ne connait pas encore. Mais c'est un poste transitoire.
Réseau Cyclades[modifier | modifier le code]
Réalisation[modifier | modifier le code]

En 1970, Louis Pouzin revoit l'un de ses camarades à Polytechnique, Michel Monpetit, qui cumule alors les fonctions de principal adjoint de Maurice Allègre au Plan Calcul et de directeur adjoint de l'IRIA, sous la direction du professeur Michel Laudet. Michel Montpetit et une délégation française viennent de visiter les États-Unis et de découvrir l'existence du réseau ARPANET. Ils veulent construire un système semblable en France et sollicitent Louis Pouzin pour en prendre la responsabilité.

En 1971, la décision politique est prise de créer Arpanet français avec l'appui industriel de la Compagnie internationale pour l'informatique (CII, prononcer C 2 i) créée en 1966 avec le Plan Calcul. Avec l'accord de Maurice Allègre, Michel Monpetit recrute Pouzin à l'IRIA. Ce dernier s'installe à Rocquencourt et constitue son équipe (Jean Le Bihan, Jean-Louis Grangé, Hubert Zimmermann, Gérard Le Lann, Jean-Pierre Touchard) pour créer un réseau capable de relier une vingtaine d'ordinateurs hétérogènes de différentes institutions de recherche et d'Universités françaises. Le cahier des charges est le suivant [5]: le réseau doit être apte :

    à la transmission de données ;
    aux applications interactives ;
    aux bases de données réparties ;
    au travail collaboratif à distance.

Le projet, baptisé Cyclades par analogie à l'archipel grec homonyme, est doté du budget, énorme pour l'époque, de 5 millions de francs par an pendant 5 ans. Dans l'esprit de ses promoteurs, il doit aider les chercheurs à travailler à distance, et susciter de nouveaux axes de recherche, mais aussi faire un contre-poids industriel à une recherche jugée trop universitaire au sein de l'IRIA de l'époque.

En 1972, l'équipe définit les spécifications du réseau, acquiert un ordinateur Mitra 15 de la CII comme cœur de réseau et se lance dans la programmation. En novembre 1973, une première démonstration est réalisée en présence des autorités scientifiques et politiques. Trois ordinateurs, l'un à l'IRIA, l'autre à la CII et le troisième à l'IMAG de Grenoble fonctionnent de concert. En 1975, le réseau Cyclades relie 25 ordinateurs répartis sur le territoire, plus un à Rome et un à Londres. En 1976, il fonctionne de manière routinière[source insuffisante].
Datagrammes vs Transpac[modifier | modifier le code]

En 1976, le CCITT, ancêtre de l'UIT-T, adopte le standard X.25, pour les services de transmission des données en mode paquets. C'est un protocole d'accès au réseau en mode circuits virtuels[8]. Ce standard reprend largement les protocoles conçus par les informaticiens du CNET et du CCETT pour le réseau expérimental RCP en 1972, puis pour le réseau public Transpac ouvert en 1978[9]. L'accord international nécessaire pour ce standard a bénéficié de la convergence d'idées avec les américains du réseau expérimental Arpanet. Ceux-ci, quand ils ont voulu dans les années 1970 passer à un réseau commercialisable, nommé Telenet, avaient eux aussi, indépendamment, choisi les circuit virtuels [10]. L'accord a également bénéficié de ce que les canadiens du réseau Datapac, après avoir un temps envisagé un protocole à 4 couches de protocole (dont une de datagramme et une au dessus pour les circuits virtuels), ont finalement préféré supprimer la couche datagramme (et avoir au passage un protocole de circuits virtuels plus simple)[11],[9].

Louis Pouzin a abondamment critiqué, y compris à l'étranger, le choix des circuits virtuels par les PTT français pour Transpac[12],[13],[14],[15]. Reconnaissant lui-même que, du point de vue technique, « il y avait des arguments des deux côtés »[13], il insistait sur d'autres types de considérations. Un de ses arguments récurrents pour discréditer les circuits virtuels est qu'ils auraient été l'œuvre d'ingénieurs de culture télécom et non pas par des informaticiens comme lui[15]. En réalité : (1) l'équipe qui avait développé RCP, puis spécifié sous son entière responsabilité Transpac, avait été dirigée par Rémi Després, informaticien depuis 1963 et PhD en informatique de Berkeley ; (2) Larry Roberts, PhD du MIT qui avait été le concepteur et directeur du réseau Arpanet[16], premier réseau à routage interne de type datagrammes, n'avait jamais travaillé pour aucun opérateur avant de préférer un protocole de circuis virtuels pour les utilisateurs de Telenet[17]. Conférencier charismatique et apprécié pour son humour, il présentait des dessins caricaturaux du genre de celui qui est reproduit dans la revue La Recherche en 2000. En 1978 (alors que c'était l'année d'ouverture de Transpac), on y voit le ministre des PTT se dire convaincu que « les transmissions de données se feront encore longtemps à bicyclette »[15].

Selon Maurice Allègre, délégué à l'informatique du Plan Calcul jusqu'en 1974 :

    « Louis Pouzin, polytechnicien et chercheur de très grand talent, (est à l'époque) venu proposer un projet de réseau maillé d'ordinateurs basé sur quelque chose de totalement nouveau : la commutation de paquets. Très vite, les recherches ont connu un plein succès, au point que j'ai déployé de grands efforts pour faire adopter le projet par la direction générale des Télécommunications comme base pour leur futur réseau de transmissions de données », poursuivait M. Allègre, « Je me suis malheureusement heurté à un mur. Nous aurions pu être parmi les pionniers du monde Internet (...) », conclut le courrier de l'ancien haut fonctionnaire, « Nous n'en sommes que des utilisateurs, fort distants des lieux où s'élabore l'avenir[18]. »

Fin de l'expérimentation[modifier | modifier le code]

Le réseau continue de fonctionner quelque temps et est finalement dissous en 1978, trois ans à peine après sa mise en service : la « mise à disposition gratuite des circuits et des adaptateurs de lignes (modems) pour une durée de 3 ans », qui avait été prévue dans l'accord « entre le Directeur général des télécommunications, Louis-Joseph Libois et le Délégué à l’informatique, Maurice Allègre, le 29 juin 1972 » [19],[20], a été prolongée de 3 ans, puis interrompue avec l'arrivée fin 1978 du réseau public Transpac, fondé sur le standard international de l'époque.

Après l'élection de Valéry Giscard d'Estaing à la présidence de la République en 1974, la Délégation à l'informatique, devenue inutile avec l'abandon du projet Unidata d'un grand constructeur européen d'ordinateurs, est supprimée. Le budget de Cyclades, hors financement des laisons télécom qui reste assuré par la DGT, est alors repris par la Direction des industries électroniques et informatiques (DIELI)[15].
Après Cyclades[modifier | modifier le code]

Après Cyclades, Louis Pouzin reste à l'IRIA jusqu'en 1980 comme directeur de projets pilotes :

    robotique (Spartacus) ;
    base de données réparties ;
    bureautique (KAYAK) confié à Najah Naffah ;
    réalisation de compilateurs du langage Pascal et d'un système d'exploitation de type Unix écrit en Pascal (SOL), dont il confie la direction à Michel Gien ;
    sureté de fonctionnement.

En 1980, Bernard Lorimy lui propose de rejoindre l'Agence de l'informatique pour diriger les projets pilotes, mais Louis Pouzin refuse et préfère offrir ses services au CNET, devenu plus tard la branche recherche de France Télécom. La direction des projets pilotes sera alors confiée à Philippe Oziard (1980-1982), puis à Robert Mahl (1983-1986).

En 1989, Louis Pouzin devient le doyen de l'Institut Theseus, un MBA orienté vers les télécommunications, créé par France Télécom à Sophia Antipolis.

Retraité, il préside le conseil d'administration du Native Language Internet Consortium (NLIC) et est chargé du développement à l'association Eurolinc, dont l'objectif est de promouvoir le multilinguisme dans l'Internet.

Depuis 2003, Louis Pouzin défend une autre gouvernance de l'internet dans le cadre du processus lancé par le SMSI. Présent à toutes les réunions et sommets (Genève, 2003 - Tunis, 2005 - Athènes, 2006) il est un des plus grands experts sur le multilinguisme dans la communauté mondiale d'internet. Il a été également président d'honneur de la Société française de l'internet. Depuis 2012, Louis Pouzin travaille à la promotion d'un nouvel internet non-IP, Recursive Internetwork Architecture (RINA), inventé en 2008 par John Day, un ancien de l'ARPANET.
Distinctions[modifier | modifier le code]
Prix[modifier | modifier le code]

En 2001, Louis Pouzin s'est vu décerner le prix Internet Award (IEEE) Internet pour sa contribution aux protocoles qui ont permis le développement de réseaux tel qu'Internet.

Louis Pouzin a été honoré par l'ISOC comme l'un des pionniers de l'internet lors de la première remise de prix du temple de la renommée d'Internet à la conférence Global INET qui s'est tenue à Genève en Suisse, le 23 avril 2012[21].

Le 18 mars 2013, Louis Pouzin reçoit le premier Queen Elizabeth Prize for Engineering[22] conjointement avec Robert Kahn, Vinton Cerf, Tim Berners-Lee et Marc Andreessen. Le prix leur est attribué pour leurs contributions majeures à la création et au développement d'internet et du World Wide Web.

Le 8 novembre 2016, Louis Pouzin a reçu en Arménie le Global IT Award, un prix attribué par un comité qui a déjà honoré Mario Mazzola, Steve Wozniak ou Eugène Kaspersky.
Décorations[modifier | modifier le code]

    Officier de la Légion d'honneur (chevalier le 19 mars 2003[23],[24] par Claudie Haigneré, ministre déléguée à la recherche et aux nouvelles technologies, puis a été promu officier par décret du 31 décembre 2018[25], remise en juin 2019 par Marc Watin-Augouard)

Notes et références[modifier | modifier le code]
Notes[modifier | modifier le code]

    ↑ Pendant sa scolarité à l'École polytechnique, Louis Pouzin fait partie de « la Khômiss dont le rôle était la constitution d'une identité "X", un groupe d'une douzaine d'élèves chargés de perpétuer le désordre et les traditions[4] » ; il en est le pitaine Téléphone.

Références[modifier | modifier le code]

    ↑ «  », sur www.economica.fr (consulté le 23 août 2020)
    ↑ «  », décembre 1974
    ↑ «  », sur bibli-aleph.polytechnique.fr (consulté le 23 août 2020)
    ↑ Louis Pouzin : L'un des pères de l'internet, p. 14-15
    ↑ a b et c « Entretien avec Louis Pouzin », par Isabelle Bellin, le 19/03/2007
    ↑ (en) How the Web was Born : The Story of the World Wide Web, p. 36 sur Google Livres
    ↑ "Le projet Cyclades (1972-1977)", par Louis Pouzin, dans Entreprises et histoire 2002/1 (no 29), pages 33 à 40
    ↑ «  »
    ↑ a et b (en) Rémi Després, «  », novembre 201
    ↑ (en) Larry Roberts, «  », décembre 1978
    ↑ (en) Tony Rybczynski, «  », décembre 2009
    ↑ (en) «  », Computerworld, 22 octobre 1975(consulté le 9 décembre 2020)
    ↑ a et b (en) Louis Pouzin, «  », Proceedings AFIPS NCC,‎ juin 1976, p. 483-494
    ↑ (en) Louis Pouzin, «  », Proceedings IFIP ICCC,‎ août 1976, p. 63-567 -
    ↑ a b c et d (en) Louis Pouzin, «  », La Recherche,‎ février 2000(lire en ligne, consulté le 9 décembre 2020)
    ↑ (en) «  »)
    ↑ «  », sur Internet Hall of Fame (consulté le 9 décembre 2020)
    ↑ Stéphane Foucart, «  », Le Monde, 5 août 2006(consulté le 2 juillet 2013)
    ↑ Valérie Schafer, , Cigref & Nuvis, 2012
    ↑ «  »
    ↑ (en) Louis Pouzin sur le Internet Hall of Fame.
    ↑ (en) 2013 QEPrize Winners
    ↑ Louis Henri Pouzin sur les Annales des Mines
    ↑ Légion d'Honneur : Louis Pouzin promu, le numérique distingué
    ↑ Décret du 31 décembre 2018 portant promotion et nomination

Voir aussi[modifier | modifier le code]

Sur les autres projets Wikimedia :

    Louis Pouzin, sur Wikimedia Commons

Bibliographie[modifier | modifier le code]

    (en) James Gillies et Robert Cailliau, , Oxford, Oxford University Press, 2007, 372 p. (ISBN 978-0-19-286207-5 et 0-19-286207-3, OCLC 603986579, présentation en ligne)

    Maryse Carmes et Jean-Marc Noyer, , Paris, Presse des Mines, coll. « Territoires numériques », 2013, 279 p. (ISBN 978-2-35671-103-8 et 2-35671-103-1, OCLC 898327208, présentation en ligne, lire en ligne)

    Jean-Baptiste Vallet, , Paris, Cherche Midi, coll. « Documents », 2015, 220 p. (ISBN 978-2-7491-4278-4 et 2-7491-4278-4, OCLC 949825777, présentation en ligne)

    Chantal Lebrument et Fabien Soyez (préf. Korben), , Paris, Economica, coll. « Collection Cyberstratégie », 2018, 170 p. (ISBN 978-2-7178-7047-3 et 2-7178-7047-4, OCLC 1078689980, présentation en ligne)

Articles connexes[modifier | modifier le code]

    Vint Cerf
    Robert Elliot Kahn

Liens externes[modifier | modifier le code]

    Notices d'autorité :
        Fichier d’autorité international virtuel
        International Standard Name Identifier
        Bibliothèque nationale de France (données)
        Système universitaire de documentation
        Bibliothèque du Congrès
        Gemeinsame Normdatei
        Bibliothèque royale des Pays-Bas
        Bibliothèque universitaire de Pologne
        WorldCat Id
        WorldCat

"""

In [33]:
%%time
input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)

input_ids = input_ids.to("cuda")["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens = 500,
    do_sample=False, # si nous passons ici à "True" les réponse changerons en fonction d'un seed
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

Louis Pouzin, un ingénieur français qui a créé et développé le Réseau Cyclades, est décédé à l'âge de 90 ans.
CPU times: user 1.82 s, sys: 1.22 ms, total: 1.82 s
Wall time: 1.88 s


In [37]:
%%time
input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)

input_ids = input_ids.to("cuda")["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens = 500,
    do_sample=True,
    num_beams=3
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

L'ingénieur français en informatique, Louis Pouzin, a été nommé Polytechnicien informaticien à la prestigieuse Légion d'honneur de l'année 2018.
CPU times: user 1.78 s, sys: 3.86 ms, total: 1.78 s
Wall time: 2.37 s


In [39]:
%%time
input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)

input_ids = input_ids.to("cuda")["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens = 500,
    no_repeat_ngram_size=2,
    do_sample=True,
    num_beams=2
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

Louis Pouzin, un ingénieur français qui a créé et développé le Réseau Cyclades, à base de datagram pur, est décédé dimanche soir lors d'une conférence de presse en Grande Bretagne.
CPU times: user 1.94 s, sys: 1.68 ms, total: 1.94 s
Wall time: 3.19 s
